# COW vs MOR 직접 비교 실험

동일한 데이터와 동일한 작업을 COW/MOR 테이블에서 각각 수행하고,  
**실행 시간, 파일 구조, 메타데이터 계층**을 정량적으로 비교합니다.

데이터 볼륨을 **50만 건**으로 설정하여,  
COW의 **파일 재작성 비용**과 MOR의 **Delete File 전략** 차이가 실행 시간에 명확히 드러나도록 합니다.

## 실험 설계

| 단계 | 작업 | 데이터 규모 | 설명 |
|------|------|------------|------|
| 0 | Warmup | — | JVM/Iceberg 초기화 비용 제거 |
| 1 | INSERT | 500,000건 | 초기 데이터 적재 (COW/MOR 동일) |
| 2 | UPDATE | 50,000건 (10%) | 상태 변경 → COW는 파일 재작성, MOR는 Delete File + 새 데이터 |
| 3 | DELETE | 20,000건 (4%) | 삭제 → COW는 파일 재작성, MOR는 Delete File만 추가 |
| 4 | SELECT | 전체 | 읽기 → COW는 데이터만, MOR는 Delete File 병합 필요 |

> **Warmup**: Spark/JVM은 첫 작업에서 클래스 로딩, JIT 컴파일 등의 초기화 비용이 발생합니다.  
> 먼저 실행되는 쪽이 불리해지는 것을 방지하기 위해, 별도의 테이블에 dummy 작업을 수행합니다.

## 환경 설정

In [2]:
import sys
sys.path.append('..')

import time
import pandas as pd

from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, count_files, total_size, show_metadata_hierarchy

In [3]:
spark = create_spark_session()

COW_TABLE = "demo.lab.cmp_cow_orders"
MOR_TABLE = "demo.lab.cmp_mor_orders"
COW_PATH = "/home/jovyan/data/warehouse/lab/cmp_cow_orders"
MOR_PATH = "/home/jovyan/data/warehouse/lab/cmp_mor_orders"

Spark + Iceberg 세션 준비 완료 (warehouse: file:///home/jovyan/data/warehouse)


## 테이블 생성

동일한 스키마, 동일한 파티셔닝 — **TBLPROPERTIES만 다릅니다**.

In [4]:
# 기존 테이블 정리
spark.sql(f"DROP TABLE IF EXISTS {COW_TABLE}")
spark.sql(f"DROP TABLE IF EXISTS {MOR_TABLE}")

# COW 테이블
spark.sql(f"""
CREATE TABLE {COW_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.delete.mode'='copy-on-write',
    'write.update.mode'='copy-on-write',
    'write.merge.mode'='copy-on-write'
)
""")
print(f"COW 테이블 생성 완료: {COW_TABLE}")

# MOR 테이블
spark.sql(f"""
CREATE TABLE {MOR_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
TBLPROPERTIES (
    'write.delete.mode'='merge-on-read',
    'write.update.mode'='merge-on-read',
    'write.merge.mode'='merge-on-read'
)
""")
print(f"MOR 테이블 생성 완료: {MOR_TABLE}")

# ── JVM Warmup ──
# 별도 dummy 테이블에 INSERT → UPDATE → DELETE → DROP 수행하여
# Spark/JVM/Iceberg 초기화 비용을 미리 소모합니다.
WARMUP_TABLE = "demo.lab.cmp_warmup"
spark.sql(f"DROP TABLE IF EXISTS {WARMUP_TABLE}")
spark.sql(f"""
CREATE TABLE {WARMUP_TABLE} (id BIGINT, val STRING)
USING ICEBERG
""")
warmup_df = spark.createDataFrame([(i, "x") for i in range(100)], ["id", "val"])
warmup_df.writeTo(WARMUP_TABLE).append()
spark.sql(f"UPDATE {WARMUP_TABLE} SET val = 'y' WHERE id < 10")
spark.sql(f"DELETE FROM {WARMUP_TABLE} WHERE id >= 90")
spark.sql(f"SELECT COUNT(*) FROM {WARMUP_TABLE}").collect()
spark.sql(f"DROP TABLE {WARMUP_TABLE}")
print("JVM Warmup 완료 — 이후 측정은 초기화 비용이 제거된 상태")

COW 테이블 생성 완료: demo.lab.cmp_cow_orders
MOR 테이블 생성 완료: demo.lab.cmp_mor_orders
JVM Warmup 완료 — 이후 측정은 초기화 비용이 제거된 상태


---
## 단계 1: INSERT 500,000건

동일한 데이터를 양쪽 테이블에 삽입합니다.  
INSERT는 COW/MOR 모두 **새 데이터 파일을 생성**하는 동일한 동작이므로, 시간 차이가 거의 없어야 합니다.

In [5]:
NUM_RECORDS = 500_000

orders = generate_orders(num_records=NUM_RECORDS, seed=42)
df = to_spark_df(spark, orders)
df.cache()
df.count()  # 캐시 실체화

# COW INSERT
start = time.time()
df.writeTo(COW_TABLE).append()
cow_insert_time = time.time() - start
print(f"COW INSERT 시간: {cow_insert_time:.3f}초")

# MOR INSERT
start = time.time()
df.writeTo(MOR_TABLE).append()
mor_insert_time = time.time() - start
print(f"MOR INSERT 시간: {mor_insert_time:.3f}초")

df.unpersist()

cow_count_after_insert = spark.sql(f"SELECT COUNT(*) FROM {COW_TABLE}").collect()[0][0]
print(f"\n양쪽 테이블 모두 {cow_count_after_insert:,}건 삽입 완료")
print(f"INSERT 시간 차이: {abs(cow_insert_time - mor_insert_time):.3f}초 (거의 동일해야 정상)")

# INSERT 후 파일 크기 확인 — 이 파일들이 COW에서 재작성 대상
cow_data_size = total_size(COW_PATH, ext=".parquet")
print(f"\n파티션당 Parquet 파일 크기 합계: {cow_data_size / 1024:.0f} KB")
print(f"→ UPDATE/DELETE 시 COW는 이 크기만큼 전체를 재작성해야 합니다")

COW INSERT 시간: 2.114초
MOR INSERT 시간: 1.198초

양쪽 테이블 모두 500,000건 삽입 완료
INSERT 시간 차이: 0.916초 (거의 동일해야 정상)

파티션당 Parquet 파일 크기 합계: 3418 KB
→ UPDATE/DELETE 시 COW는 이 크기만큼 전체를 재작성해야 합니다


### 관찰 포인트 — INSERT

INSERT는 COW/MOR 차이가 없습니다. 둘 다 **새 데이터 파일을 생성**할 뿐이므로, Warmup 이후에는 시간이 거의 동일합니다.

- COW: 새 Parquet 파일 생성
- MOR: 새 Parquet 파일 생성
- **차이점 없음** — 기존 데이터를 수정하지 않으므로 전략이 개입하지 않음

---
## 단계 2: UPDATE 50,000건 (전체의 10%)

order_id 기준으로 처음 50,000건의 상태를 변경합니다.  
**여기서부터 COW와 MOR의 차이가 드러납니다.**

- **COW**: 50,000건이 속한 파티션의 데이터 파일을 **통째로 재작성** — 나머지 450,000건도 다시 씀
- **MOR**: 50,000건의 **Positional Delete File** + 변경된 값의 **새 데이터 파일**만 추가

In [6]:
UPDATE_COUNT = 50_000

# COW UPDATE
start = time.time()
spark.sql(f"UPDATE {COW_TABLE} SET status = 'refunded' WHERE order_id <= {UPDATE_COUNT}")
cow_update_time = time.time() - start
print(f"COW UPDATE 시간: {cow_update_time:.3f}초")

# MOR UPDATE
start = time.time()
spark.sql(f"UPDATE {MOR_TABLE} SET status = 'refunded' WHERE order_id <= {UPDATE_COUNT}")
mor_update_time = time.time() - start
print(f"MOR UPDATE 시간: {mor_update_time:.3f}초")

ratio = cow_update_time / mor_update_time if mor_update_time > 0 else float('inf')
print(f"\nCOW/MOR 비율: {ratio:.2f}x — COW가 {ratio:.1f}배 느림")
print(f"({UPDATE_COUNT:,}건 변경, COW는 {NUM_RECORDS:,}건 분량의 파일을 재작성)")

COW UPDATE 시간: 2.000초
MOR UPDATE 시간: 1.472초

COW/MOR 비율: 1.36x — COW가 1.4배 느림
(50,000건 변경, COW는 500,000건 분량의 파일을 재작성)


### 관찰 포인트 — UPDATE

| | COW | MOR |
|---|---|---|
| **동작** | 영향받은 파티션의 데이터 파일을 통째로 재작성 | Delete File + 새 데이터 파일만 추가 |
| **I/O** | 50,000건 변경인데 500,000건 분량을 기록 → **쓰기 증폭** | 변경된 50,000건 분량만 기록 → 최소 I/O |
| **결과 파일** | 기존 파일 DELETED → 새 파일 ADDED | 기존 파일 유지 + Delete File ADDED + 새 데이터 ADDED |

> 50만 건 규모에서는 COW의 **파일 재작성 비용**이 실행 시간에 명확히 반영됩니다.  
> 3개 파티션 파일(각 수 MB)을 통째로 재작성하는 COW vs 작은 Delete File만 추가하는 MOR.

---
## 단계 3: DELETE 20,000건 (전체의 4%)

order_id 50,001~70,000번 주문을 삭제합니다.

- **COW**: 해당 행이 빠진 **새 데이터 파일을 재작성** (나머지 행도 다시 씀)
- **MOR**: 삭제 위치만 기록한 **Positional Delete File**만 추가

In [7]:
DELETE_COUNT = 20_000

# COW DELETE
start = time.time()
spark.sql(f"""
    DELETE FROM {COW_TABLE}
    WHERE order_id BETWEEN {UPDATE_COUNT + 1} AND {UPDATE_COUNT + DELETE_COUNT}
""")
cow_delete_time = time.time() - start
print(f"COW DELETE 시간: {cow_delete_time:.3f}초")

# MOR DELETE
start = time.time()
spark.sql(f"""
    DELETE FROM {MOR_TABLE}
    WHERE order_id BETWEEN {UPDATE_COUNT + 1} AND {UPDATE_COUNT + DELETE_COUNT}
""")
mor_delete_time = time.time() - start
print(f"MOR DELETE 시간: {mor_delete_time:.3f}초")

ratio = cow_delete_time / mor_delete_time if mor_delete_time > 0 else float('inf')
print(f"\nCOW/MOR 비율: {ratio:.2f}x — COW가 {ratio:.1f}배 느림")
print(f"({DELETE_COUNT:,}건 삭제, COW는 전체 파티션 파일을 재작성)")

COW DELETE 시간: 2.313초
MOR DELETE 시간: 1.769초

COW/MOR 비율: 1.31x — COW가 1.3배 느림
(20,000건 삭제, COW는 전체 파티션 파일을 재작성)


### 관찰 포인트 — DELETE

| | COW | MOR |
|---|---|---|
| **동작** | 삭제 행이 빠진 새 파일 재작성 | Positional Delete File만 추가 |
| **I/O** | 20,000건 삭제인데 전체 파일 재작성 | 삭제 위치(file_path + pos)만 기록 → **매우 작은 I/O** |
| **결과** | 기존 데이터 파일 DELETED → 새 파일 ADDED | 기존 데이터 파일 유지 + Delete File ADDED |

> DELETE는 UPDATE보다 MOR의 이점이 더 큽니다.  
> UPDATE는 새 값을 기록한 데이터 파일이 필요하지만, DELETE는 **Delete File만으로 충분**합니다.

---
## 단계 4: SELECT — 읽기 성능 비교

전체 데이터를 읽는 시간을 비교합니다.

- **COW**: 데이터 파일만 읽으면 됨 (이미 최신 상태)
- **MOR**: 데이터 파일 + Delete File을 **병합(merge)**해야 정확한 결과를 얻음

In [8]:
# COW SELECT
start = time.time()
cow_count = spark.sql(f"SELECT COUNT(*) FROM {COW_TABLE}").collect()[0][0]
cow_read_time = time.time() - start
print(f"COW SELECT 시간: {cow_read_time:.3f}초 (레코드 수: {cow_count:,})")

# MOR SELECT
start = time.time()
mor_count = spark.sql(f"SELECT COUNT(*) FROM {MOR_TABLE}").collect()[0][0]
mor_read_time = time.time() - start
print(f"MOR SELECT 시간: {mor_read_time:.3f}초 (레코드 수: {mor_count:,})")

ratio = mor_read_time / cow_read_time if cow_read_time > 0 else float('inf')
print(f"\nMOR/COW 비율: {ratio:.2f}x")
print(f"레코드 수 일치 여부: {cow_count == mor_count} ({cow_count:,}건)")

COW SELECT 시간: 0.145초 (레코드 수: 480,000)
MOR SELECT 시간: 0.288초 (레코드 수: 480,000)

MOR/COW 비율: 1.99x
레코드 수 일치 여부: True (480,000건)


### 관찰 포인트 — SELECT

| | COW | MOR |
|---|---|---|
| **읽기 대상** | 데이터 파일만 | 데이터 파일 + Delete File 병합 |
| **Merge 비용** | 없음 | Delete File 기반 필터링 필요 |
| **특성** | 쓰기 시 이미 최적화 완료 | 읽기 시 병합 비용 발생 |

> 소규모 데이터에서는 읽기 차이가 미미하지만, Delete File이 누적될수록 MOR의 읽기 비용이 증가합니다.  
> 이를 해결하기 위해 **정기적인 컴팩션(compaction)**이 필요합니다 (→ 4_optimization 모듈).

---
## 파일 시스템 비교

시간 측정만으로는 **왜** 차이가 나는지 알기 어렵습니다.  
파일 시스템 수준에서 **데이터 파일 수, Delete File 유무, 메타데이터 계층**을 직접 비교합니다.

In [9]:
print("=" * 60)
print("COW 테이블 파일 구조:")
print("=" * 60)
show_tree(COW_PATH)

cow_data_files = count_files(COW_PATH, ext=".parquet")
cow_total = total_size(COW_PATH)

print(f"\nParquet 파일: {cow_data_files}개, 총 크기: {cow_total:,} bytes")

print("\n" + "=" * 60)
print("MOR 테이블 파일 구조:")
print("=" * 60)
show_tree(MOR_PATH)

mor_data_files = count_files(MOR_PATH, ext=".parquet")
mor_total = total_size(MOR_PATH)

print(f"\nParquet 파일: {mor_data_files}개, 총 크기: {mor_total:,} bytes")

COW 테이블 파일 구조:
├── data/
│   ├── order_date_month=2024-01/
│   │   ├── 00001-25-47934f7c-9286-4223-93db-bc2af0da7f84-00001.parquet  (1.1 MB)
│   │   ├── 00001-40-8762e0fa-f3ed-41ca-ac9e-4d6e7cccf513-00001.parquet  (1.1 MB)
│   │   └── 00001-47-342c099a-7482-4b17-ba40-441433059299-00001.parquet  (1.1 MB)
│   ├── order_date_month=2024-02/
│   │   ├── 00002-26-47934f7c-9286-4223-93db-bc2af0da7f84-00001.parquet  (1.1 MB)
│   │   ├── 00002-41-8762e0fa-f3ed-41ca-ac9e-4d6e7cccf513-00001.parquet  (1.1 MB)
│   │   └── 00002-48-342c099a-7482-4b17-ba40-441433059299-00001.parquet  (1.0 MB)
│   └── order_date_month=2024-03/
│       ├── 00000-24-47934f7c-9286-4223-93db-bc2af0da7f84-00001.parquet  (1.1 MB)
│       ├── 00000-39-8762e0fa-f3ed-41ca-ac9e-4d6e7cccf513-00001.parquet  (1.1 MB)
│       └── 00000-46-342c099a-7482-4b17-ba40-441433059299-00001.parquet  (1.1 MB)
└── metadata/
    ├── 33eff80a-ec43-44cf-a6b4-7e0f178788b1-m0.avro  (7.3 KB)
    ├── 7819d505-28d8-47a7-8484-68bff6ac517d-m0.avro  (7.3

### 데이터 파일 vs Delete File 비교

`.files` 메타데이터 테이블로 **현재 활성 상태의 파일**을 비교합니다.  
COW는 데이터 파일만, MOR는 데이터 파일 + Delete File이 혼재합니다.

In [10]:
# COW — 활성 파일 통계
cow_files = spark.sql(f"""
    SELECT content,
           COUNT(*) AS file_count,
           SUM(record_count) AS total_records,
           SUM(file_size_in_bytes) AS total_bytes
    FROM {COW_TABLE}.files
    GROUP BY content
    ORDER BY content
""").collect()

print("COW 활성 파일:")
for row in cow_files:
    content_type = "DATA" if row["content"] == 0 else "DELETES"
    print(f"  {content_type}: {row['file_count']}개 파일, "
          f"{row['total_records']}행, "
          f"{row['total_bytes'] / 1024:.1f} KB")

# MOR — 활성 파일 통계
mor_files = spark.sql(f"""
    SELECT content,
           COUNT(*) AS file_count,
           SUM(record_count) AS total_records,
           SUM(file_size_in_bytes) AS total_bytes
    FROM {MOR_TABLE}.files
    GROUP BY content
    ORDER BY content
""").collect()

print("\nMOR 활성 파일:")
for row in mor_files:
    content_type = "DATA" if row["content"] == 0 else "DELETES"
    print(f"  {content_type}: {row['file_count']}개 파일, "
          f"{row['total_records']}행, "
          f"{row['total_bytes'] / 1024:.1f} KB")

# 비교 요약
cow_data_count = sum(r["file_count"] for r in cow_files if r["content"] == 0)
cow_delete_count = sum(r["file_count"] for r in cow_files if r["content"] == 1)
mor_data_count = sum(r["file_count"] for r in mor_files if r["content"] == 0)
mor_delete_count = sum(r["file_count"] for r in mor_files if r["content"] == 1)

print(f"\n{'─' * 50}")
print(f"{'':15s} {'COW':>10s} {'MOR':>10s}")
print(f"{'─' * 50}")
print(f"{'Data 파일':15s} {cow_data_count:>10d} {mor_data_count:>10d}")
print(f"{'Delete 파일':15s} {cow_delete_count:>10d} {mor_delete_count:>10d}")
print(f"{'총 파일':15s} {cow_data_count + cow_delete_count:>10d} {mor_data_count + mor_delete_count:>10d}")
print(f"{'─' * 50}")

COW 활성 파일:
  DATA: 3개 파일, 480000행, 3271.2 KB

MOR 활성 파일:
  DATA: 6개 파일, 550000행, 3759.3 KB
  DELETES: 6개 파일, 70000행, 126.0 KB

──────────────────────────────────────────────────
                       COW        MOR
──────────────────────────────────────────────────
Data 파일                  3          6
Delete 파일                0          6
총 파일                     3         12
──────────────────────────────────────────────────


### 메타데이터 계층 비교

`metadata.json → Manifest List → Manifest File → Data File` 체인을 통해  
COW와 MOR이 **내부 메타데이터를 어떻게 다르게 구성하는지** 확인합니다.

핵심 차이:
- **COW**: DATA Manifest만 존재 (Delete File이 없으므로)
- **MOR**: DATA Manifest + **DELETES Manifest**가 별도로 존재

In [11]:
print("=" * 60)
print("COW 메타데이터 계층:")
print("=" * 60)
show_metadata_hierarchy(COW_PATH)

print("\n" + "=" * 60)
print("MOR 메타데이터 계층:")
print("=" * 60)
show_metadata_hierarchy(MOR_PATH)

COW 메타데이터 계층:
v4.metadata.json  (operation: overwrite)
│
└─▶ snap-2967345037190015042-1-7819d505-28d8-47a7-8484-68bff6ac517d.avro  [Manifest List]
    ├─▶ 7819d505-28d8-47a7-8484-68bff6ac517d-m1.avro  [Manifest — DATA: 3 ADDED]
        │   ├── data/warehouse/lab/cmp_cow_orders/data/order_date_month=2024-03/00000-46-342c099a-7482-4b17-ba40-441433059299-00001.parquet  (163507행, ADDED)
        │   ├── data/warehouse/lab/cmp_cow_orders/data/order_date_month=2024-01/00001-47-342c099a-7482-4b17-ba40-441433059299-00001.parquet  (163392행, ADDED)
        │   └── data/warehouse/lab/cmp_cow_orders/data/order_date_month=2024-02/00002-48-342c099a-7482-4b17-ba40-441433059299-00001.parquet  (153101행, ADDED)
    └─▶ 7819d505-28d8-47a7-8484-68bff6ac517d-m0.avro  [Manifest — DATA: 3 DELETED]
            ├── data/warehouse/lab/cmp_cow_orders/data/order_date_month=2024-03/00000-39-8762e0fa-f3ed-41ca-ac9e-4d6e7cccf513-00001.parquet  (170287행, DELETED)
            ├── data/warehouse/lab/cmp_cow_orders/data/

### 메타데이터 구조 차이 정리

```
COW의 Manifest List:                   MOR의 Manifest List:
├─▶ Manifest (DATA)  ← DELETE용       ├─▶ Manifest (DATA)      ← INSERT 원본
│   └── 새 파일 (ADDED)                │   └── 원본 파일 (EXISTING)
├─▶ Manifest (DATA)  ← UPDATE용       ├─▶ Manifest (DATA)      ← UPDATE 새 데이터
│   └── 새 파일 (ADDED)                │   └── 새 파일 (ADDED)
└─▶ Manifest (DATA)  ← INSERT 원본     ├─▶ Manifest (DELETES)   ← UPDATE용 Delete File
    └── 원본 파일 (EXISTING/DELETED)    │   └── delete-*.parquet (ADDED)
                                      └─▶ Manifest (DELETES)   ← DELETE용 Delete File
                                           └── delete-*.parquet (ADDED)
```

| 특성 | COW | MOR |
|------|-----|-----|
| Manifest 종류 | DATA만 | DATA + DELETES |
| 기존 파일 상태 | UPDATE/DELETE 시 DELETED로 표시 | EXISTING 유지 (삭제되지 않음) |
| 새 파일 | 전체 데이터 재작성 파일 | 변경분 데이터 파일 + Delete File |
| Manifest 크기 | 재작성된 파일 정보 포함 | 원본 유지 + 작은 추가 파일 정보 |

### COW의 "DELETED" 상태 이해하기

  Delete File vs DELETED 상태

  COW는 Delete File을 사용하지 않습니다. 하지만 매니페스트에서 DELETED 상태가 보입니다. 이 둘은 완전히 다른 개념입니다.
```
  ┌─────────┬────────────────────────────────┬────────────────────────────────────────────────────────┐
  │         │     Delete File (MOR 전용)     │             DELETED 상태 (Manifest entry)              │
  ├─────────┼────────────────────────────────┼────────────────────────────────────────────────────────┤
  │ 정체    │ -deletes.parquet 파일          │ 매니페스트 엔트리의 상태 플래그                        │
  ├─────────┼────────────────────────────────┼────────────────────────────────────────────────────────┤
  │ 의미    │ "이 행들을 읽을 때 건너뛰어라" │ "이 데이터 파일은 더 이상 현재 스냅샷에 속하지 않는다" │
  ├─────────┼────────────────────────────────┼────────────────────────────────────────────────────────┤
  │ COW에서 │ 사용 안 함                     │ 사용함 — 재작성으로 대체된 옛 파일을 표시              │
  └─────────┴────────────────────────────────┴────────────────────────────────────────────────────────┘
```

  COW가 UPDATE를 실행하면:
  1. 기존 파티션 파일을 읽고, 변경 사항을 반영한 새 파일을 작성한다 → ADDED
  2. 옛 파일은 새 파일로 대체되었으므로 → DELETED로 표시

```
  v3(UPDATE) manifest list:
  ├─▶ m1.avro  [DATA: 3 ADDED]     ← 새로 재작성된 데이터 파일 3개
  └─▶ m0.avro  [DATA: 3 DELETED]   ← 대체된 옛 데이터 파일 3개 (디스크에서 삭제되지는 않음)
```

  DELETED는 **"이 파일이 테이블의 현재 뷰에서 제거됨"**이라는 메타데이터 상태일 뿐, 디스크에서 물리 삭제되는 것이 아닙니다. 이전 스냅샷에서 Time Travel로 여전히 접근할 수 있습니다.

  ---
  metadata 디렉토리 vs 현재 스냅샷

  metadata 디렉토리에는 모든 스냅샷의 manifest가 누적되어 있지만, 현재 스냅샷은 그 중 일부만 참조합니다.

```
  metadata 디렉토리 (전체):              현재 스냅샷(v4)이 참조하는 것:
  ├── 33eff80a-m0.avro  ← v2(INSERT)용    (참조 안 함 — 이미 대체됨)
  ├── 7819d505-m0.avro  ← v3(UPDATE)용    (참조 안 함 — 이미 대체됨)
  ├── 7819d505-m1.avro  ← v3(UPDATE)용    (참조 안 함 — 이미 대체됨)
  ├── 9c1c6bb8-m0.avro  ← v4(DELETE)용    ✅ 3 DELETED
  └── 9c1c6bb8-m1.avro  ← v4(DELETE)용    ✅ 3 ADDED
```

  나머지 manifest 파일은 이전 스냅샷이 Time Travel용으로 참조하는 파일입니다.

  ---
  DELETED manifest가 "하나"인 이유

  DELETED manifest 수는 교체 대상 파일이 몇 개의 manifest에 걸쳐 있었느냐에 따라 결정됩니다.

  INSERT를 한 번에 했으므로, 3개 파티션 파일이 manifest 1개에 전부 들어 있었습니다:

```
  v2(INSERT):
  └─▶ 33eff80a-m0.avro
        ├── partition-01.parquet (ADDED)
        ├── partition-02.parquet (ADDED)
        └── partition-03.parquet (ADDED)
```

  UPDATE가 이 3개를 모두 교체할 때, 원본 manifest 1개만 재작성하면 됩니다:

```
  v3(UPDATE):
  ├─▶ m1.avro  ← 새 파일 3개 (ADDED)
  └─▶ m0.avro  ← 33eff80a-m0을 재작성: 옛 파일 3개 (DELETED)
```

  만약 INSERT를 3번에 나눠서 했다면, manifest가 3개 생기고, UPDATE 시 DELETED manifest도 3개가 됩니다:

  만약 INSERT를 3번에 나눠 했다면:
```
  v3(UPDATE):
  ├─▶ m3.avro  ← 새 파일 (ADDED)
  ├─▶ m2.avro  ← 배치3 manifest 재작성 (DELETED)
  ├─▶ m1.avro  ← 배치2 manifest 재작성 (DELETED)
  └─▶ m0.avro  ← 배치1 manifest 재작성 (DELETED)
```

  DELETED manifest 수 = 교체 대상 파일이 흩어져 있던 manifest 수

---
## 종합 비교표

In [12]:
comparison = pd.DataFrame({
    '항목': [
        'INSERT 시간 (초)',
        'UPDATE 시간 (초)',
        'DELETE 시간 (초)',
        'SELECT 시간 (초)',
        '활성 Data 파일 수',
        '활성 Delete 파일 수',
        '총 활성 파일 수',
        '디스크 총 크기 (KB)',
        '최종 레코드 수',
    ],
    'COW': [
        f"{cow_insert_time:.3f}",
        f"{cow_update_time:.3f}",
        f"{cow_delete_time:.3f}",
        f"{cow_read_time:.3f}",
        cow_data_count,
        cow_delete_count,
        cow_data_count + cow_delete_count,
        f"{cow_total / 1024:.1f}",
        f"{cow_count:,}",
    ],
    'MOR': [
        f"{mor_insert_time:.3f}",
        f"{mor_update_time:.3f}",
        f"{mor_delete_time:.3f}",
        f"{mor_read_time:.3f}",
        mor_data_count,
        mor_delete_count,
        mor_data_count + mor_delete_count,
        f"{mor_total / 1024:.1f}",
        f"{mor_count:,}",
    ],
})

print("=" * 65)
print(f"COW vs MOR 종합 비교 ({NUM_RECORDS:,}건, UPDATE {UPDATE_COUNT:,}건, DELETE {DELETE_COUNT:,}건)")
print("=" * 65)
print(comparison.to_string(index=False))

COW vs MOR 종합 비교 (500,000건, UPDATE 50,000건, DELETE 20,000건)
            항목     COW     MOR
 INSERT 시간 (초)   2.114   1.198
 UPDATE 시간 (초)   2.000   1.472
 DELETE 시간 (초)   2.313   1.769
 SELECT 시간 (초)   0.145   0.288
  활성 Data 파일 수       3       6
활성 Delete 파일 수       0       6
     총 활성 파일 수       3      12
 디스크 총 크기 (KB) 10157.0  3940.0
      최종 레코드 수 480,000 480,000


---
## 비교 결과 분석

### 1. INSERT — 차이 없음
INSERT는 새 데이터 파일을 생성하는 동일한 동작입니다.  
COW/MOR 전략은 **기존 데이터를 수정할 때만** 개입합니다.

### 2. UPDATE/DELETE — 쓰기 성능에서 MOR 우위
| 관점 | COW | MOR |
|------|-----|-----|
| **쓰기 I/O** | 50,000건 변경 → 500,000건 분량 재작성 (10x 증폭) | 50,000건 분량만 기록 |
| **파일 생성** | 기존 파일 크기와 비슷한 새 파일 (수 MB) | 작은 Delete File (~KB) + 변경분 데이터 |
| **메타데이터** | 기존 파일 DELETED → 새 파일 ADDED | 기존 파일 유지 + Delete File ADDED |

**핵심 인사이트**: 데이터 파일이 수 MB 규모가 되면, COW의 **쓰기 증폭(write amplification)** 비용이 실행 시간에 명확히 반영됩니다.  
프로덕션에서 파일이 수백 MB ~ 수 GB인 경우 이 차이는 **수십 배**로 벌어집니다.

### 3. SELECT — 읽기 성능에서 COW 우위
COW 테이블은 항상 최신 상태의 데이터 파일만 존재하므로, 추가 병합 없이 읽을 수 있습니다.  
MOR 테이블은 Delete File을 데이터 파일과 병합해야 하므로, Delete File이 누적될수록 읽기 비용이 증가합니다.

### 4. 파일 구조 — 근본적 차이
```
COW:  데이터 변경 → 파일 재작성 → 항상 "깨끗한" 데이터 파일만 존재
MOR:  데이터 변경 → Delete File 추가 → 시간이 지나면 Delete File 누적 → 컴팩션 필요
```

> **트레이드오프 요약**: COW는 **쓰기 시 비용을 지불**하고, MOR는 **읽기 시 비용을 지불**합니다.  
> MOR의 읽기 비용은 **컴팩션으로 관리**할 수 있으므로, 변경이 빈번한 워크로드에서는 MOR이 유리합니다.

---
## 선택 기준 가이드

| 워크로드 | 추천 | 이유 |
|---------|------|------|
| 읽기 중심, 수정 적음 | **COW** | 읽기 최적화, Delete File 없음 |
| 쓰기 중심, 수정 빈번 | **MOR** | 쓰기 빠름, 정기 컴팩션으로 읽기 비용 관리 |
| 대용량 파티션 + 소량 수정 | **MOR** | COW의 쓰기 증폭이 극대화되는 시나리오 |
| 실시간 분석 (짧은 지연 허용 불가) | **COW** | 읽기 시 병합 비용 없음 |
| 혼합 워크로드 | **작업별 혼합 설정** | DELETE/UPDATE/MERGE를 개별 설정 가능 |

### 실무 팁

- Iceberg에서는 **테이블 속성을 통해 작업별로 다른 전략을 설정**할 수 있습니다
  ```sql
  -- DELETE는 MOR, UPDATE는 COW로 혼합 설정
  ALTER TABLE my_table SET TBLPROPERTIES (
      'write.delete.mode'='merge-on-read',
      'write.update.mode'='copy-on-write'
  )
  ```
- MOR 테이블은 Delete File이 누적되므로, **정기적인 컴팩션(compaction)**이 필요합니다
- 컴팩션은 Delete File을 데이터 파일에 병합하여 읽기 성능을 회복시킵니다
- 프로덕션에서는 **Spark의 `rewrite_data_files`** 프로시저로 컴팩션을 수행합니다

---
## 다음 단계

> MOR 테이블의 읽기 성능을 회복하려면 **정기적인 컴팩션**이 필요합니다.  
> 이 내용은 **4_optimization** 모듈에서 자세히 다룹니다.
>
> 컴팩션을 통해 Delete File을 데이터 파일에 병합하면,  
> MOR의 쓰기 장점을 유지하면서 읽기 성능도 관리할 수 있습니다.

In [13]:
spark.stop()
print("Spark 세션 종료")

Spark 세션 종료
